In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 280)

In [2]:
data_dir = '/Users/konstantinivanov/Documents/work/Tele2Hack/DATA'

In [3]:
train_data = pd.read_csv(data_dir+'/train_data.tsv', delimiter='\t', header=None)

In [4]:
train_data.head()

,0,1,2
0,Добрый вечер Интересует сотрудничество с вашим брендом мы занимаемся продвижением артистов в вк и можем сделать вам рекламную интеграцию product placement на аудиторию более 45 000 000 подписчиков будет размещено в более 1 500 различных сообществ Проведем фотосессии ваш...,Олег здравствуйте Сообщите вашу контактную информацию Мы передадим ее коллегам Если их заинтересует ваше предложение с вами обязательно свяжутся,1
1,Добрый вечер Интересует сотрудничество с вашим брендом мы занимаемся продвижением артистов в вк и можем сделать вам рекламную интеграцию product placement на аудиторию более 45 000 000 подписчиков будет размещено в более 1 500 различных сообществ Проведем фотосессии ваш...,Клим здравствуйте По вашему населенному пункту не гарантировано покрытие 4G U 0001F614 Напишите пожалуйста ваш номер а также следующие данные 1 Марка и модель вашего телефона 2 Этаж 3 Направления окон во двор или подскажите название улицы на которую они выходят,0
2,Добрый вечер Интересует сотрудничество с вашим брендом мы занимаемся продвижением артистов в вк и можем сделать вам рекламную интеграцию product placement на аудиторию более 45 000 000 подписчиков будет размещено в более 1 500 различных сообществ Проведем фотосессии ваш...,Здравствуйте Мария Чтобы мы смогли проверить ограничения уточните следующие данные 1 Номер телефона на котором пропадает сеть 2 Как давно наблюдается проблема 3 Последний пример поиска сети дата и время 4 Если ранее были успешные звонки в данном районе укажите...,0
3,Добрый вечер Интересует сотрудничество с вашим брендом мы занимаемся продвижением артистов в вк и можем сделать вам рекламную интеграцию product placement на аудиторию более 45 000 000 подписчиков будет размещено в более 1 500 различных сообществ Проведем фотосессии ваш...,Артемий здравствуйте Это можно исправить U 0001F60A Уточните ваш регион Расскажем об условиях подключения,0
4,Добрый вечер Интересует сотрудничество с вашим брендом мы занимаемся продвижением артистов в вк и можем сделать вам рекламную интеграцию product placement на аудиторию более 45 000 000 подписчиков будет размещено в более 1 500 различных сообществ Проведем фотосессии ваш...,xxxxx уточните какая ситуация возникает со связью шумы помехи обрывы при разговорах поиск сети не можете совершить исходящий вызов нет входящей связи Сообщите пожалуйста свою локацию город улица и номер дома и номер телефона нам в ЛС Ситуация в помещении или н...,0


In [5]:
pd.isnull(train_data).sum()

0    231
1      0
2      0
dtype: int64

In [6]:
train_data.shape

(339279, 3)

In [7]:
vec_ans = list(train_data[train_data[2]==1][1].reset_index(drop=True))

In [8]:
len(vec_ans)

30844

In [9]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clusterizer', DBSCAN(eps=0.8,min_samples=10, leaf_size=30))
])

In [10]:
#n_data = 10000
n_data = len(vec_ans)

db_ans = pipeline.fit(vec_ans[:n_data])

In [11]:
db_labels = db_ans.named_steps['clusterizer'].labels_

In [12]:
val_cnt = pd.Series(db_labels).value_counts()

In [13]:
val_cnt

-1      21368
 1       2634
 3        763
 6        369
 24       275
 17       257
 11       233
 110      200
 2        188
 79       161
 30       156
 19       148
 32       131
 45       121
 123      106
 38       101
 109       99
 7         93
 15        84
 119       71
 125       68
 16        63
 4         63
 12        62
 124       61
 133       61
 63        58
 29        57
 46        56
 52        55
 87        54
 73        52
 50        47
 8         40
 5         40
 36        38
 18        37
 54        37
 35        37
 58        37
 86        36
 75        36
 172       35
 28        33
 26        31
 53        31
 83        30
 68        30
 151       29
 140       28
 118       28
 78        27
 153       27
 101       27
 47        27
 69        27
 74        26
 9         25
 107       25
 76        24
 25        23
 122       23
 147       23
 91        23
 57        23
 95        22
 43        22
 103       22
 128       22
 34        22
 84        22
 93   

In [14]:
val_cnt_ind = val_cnt.head(31).tail(30)

In [15]:
num_pop_lab = list(val_cnt_ind.index)

In [16]:
len(num_pop_lab)

30

In [17]:
vec_pop_ans = []

for label in num_pop_lab:
    for itr, labels in enumerate(db_labels):
        if label == labels:
            vec_pop_ans.append(vec_ans[itr])
            break

In [18]:
vec_pop_ans

['Алексей  здравствуйте  Сообщите 1 ю  3 ю и 5 ю цифры номера паспорта владельца SIM карты ',
 'Владимир  здравствуйте  Чем я могу вам помочь   U 0001F60A ',
 'Олег  здравствуйте  Чтобы обменять минуты на гигабайты необходимо набрать команду  155 62 кол во минут для обмена  По команде  155 77    запрос доступного для обмена количества минут  Менять можно  минуты из основного пакета  минуты из дополнительных микро пакетов  минуты из перенесенных остатков   100 минут   1 ГБ или 1 минута   10 24 МБ  Услуга доступна на тарифах линейки  Мой   кроме тарифа  Мой Tele2  Получилось   Что касается сайта  ВКонтакте  Это происходит со всеми страницами  Вы пробовал',
 'Юрий  здравствуйте  Действительно  сейчас могут быть некоторые ограничения связи в этом районе  Наши специалисты прилагают все усилия  чтобы в ближайшее время вы снова смогли пользоваться сервисом в полном объеме  Возможно  у вас остались еще вопросы ',
 'Здравствуйте  Денис  Подписок на данный момент на номере нет  Запрет на подключ

In [19]:
pd.DataFrame.to_excel(pd.DataFrame({'ans':vec_pop_ans}), data_dir+'/vec_pop_ans.xlsx',index=False)